In [1]:
from glob import glob
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import periodogram
from scipy.stats import skew, moment, kurtosis
from matplotlib import pyplot as plt
import math
from statsmodels.tsa.stattools import acf 

In [8]:
def parser_apple(aDate, aMicro):
    if not pd.isna(aDate):
        full_date = datetime.fromisoformat(aDate)
        full_date = full_date.replace(microsecond=int(aMicro))
    else :
        full_date = np.NaN
    return full_date

In [36]:
def read_apple(data_path):
    # First check if Apple Data exists in the file
    file = open(data_path, 'r')
    apple_exist = False
    if "Apple" in file.readline():
        #print("Apple Data Found")
        apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()
        apple_data = apple_data.rename(columns={apple_data.columns[0]: 'Apple Time'})
        apple_data = apple_data.sort_values(by='Apple Time')
        apple_data['Apple Time'] = apple_data['Apple Time'].apply(lambda x : int(x.timestamp()*1000))
        start = apple_data.iloc[0,0]
        apple_data["Apple Time"] = apple_data['Apple Time'].apply(lambda x: x - start)
        apple_data = apple_data.values
    else:
        #print("No Apple Data")
        apple_data = None
        start = None
    file.close()
    return apple_data, start

In [18]:
def read_garmin(data_path):
    # First check if Garmin Data exists in the file
    file = open(data_path, 'r')
    Garmin_exist = False
    if "Garmin" in file.readline():
        #print("Garmin Data Found")
        garmin_data = pd.read_csv(data_path, usecols=['Garmin Time', 'Garmin X', 'Garmin Y', 'Garmin Z', 'Garmin Magnitude'], parse_dates=[0], infer_datetime_format=True).dropna()
        garmin_data['Garmin Time'] = garmin_data['Garmin Time'].apply(lambda x : int(x.timestamp()*1000))
        start = garmin_data.iloc[0,0]
        garmin_data["Garmin Time"] = garmin_data['Garmin Time'].apply(lambda x: x - start)
        garmin_data = garmin_data.values
    else:
        #print("No Garmin Data")
        garmin_data = None
        start = None
    file.close()
    return garmin_data, start

In [10]:
def read_fitbit(data_path):
    # First check if Fitbit Data exists in the file
    file = open(data_path, 'r')
    fitbit_exist = False
    if "Fitbit" in file.readline():
        #print("Fitbit Data Found")
        fitbit_data = pd.read_csv(data_path, usecols=['Fitbit Time', 'Fitbit Second Fraction', 'Fitbit X', 'Fitbit Y', 'Fitbit Z', 'Fitbit Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()
        if fitbit_data.shape[0] > 0:
            fitbit_data = fitbit_data.rename(columns={fitbit_data.columns[0]: 'Fitbit Time'})
            fitbit_data = fitbit_data.sort_values(by='Fitbit Time')
            fitbit_data['Fitbit Time'] = fitbit_data['Fitbit Time'].apply(lambda x : int(x.timestamp()*1000))
            start = fitbit_data.iloc[0,0]
            fitbit_data["Fitbit Time"] = fitbit_data['Fitbit Time'].apply(lambda x: x - start)
            fitbit_data = fitbit_data.values
        else:
            fitbit_data = None
            start = None
    else:
        #print("No Fitbit Data")
        fitbit_data = None
        start = None
    file.close()
    return fitbit_data, start

In [3]:
# This function calculates the dominant acceleration frequency and returns that frequency and it's power
def dom_freq(aWindow):
    # Subtract the mean from signal to get rid of DC offset
    no_dc = aWindow - np.mean(aWindow)
    # Use periodogram to calculate the power of each frequency
    freq, power = periodogram(no_dc, 50) # 50 is the sampling rate of the Apple Watches Accelerometer
    # Select the dominant frequency and power
    max_index = np.argmax(power)
    dom = freq[max_index]
    max_pow = power[max_index]
    return dom, max_pow

In [4]:
def accel_metrics(some_data, start_time, out_path):
    # Start and stop of daylight savings for 2022
    ds_begin = datetime(year=2022, month=3, day=13)
    ds_end = datetime(year=2022, month=11, day=6)
    
    win_size = 31 * 1000 # Window should be 31 seconds long. Apple Time is in millisecond
    # Initialize beginning of first window as 0
    win_start = 0
    win_end = 31000

    # Initialize hash table to store indicies of each second since the sample rate
    # Isn't perfect
    win_start_table = {0: 0}
    cur_sec = 1000
    # Initialize array for output
    output = np.zeros([int(some_data[-1,0]//1000)-30, 21])
    output_iter = 0
    for i in range(some_data.shape[0] - 1):
        # CHeck if a second has passed. If so, update win_start_table
        if some_data[i, 0] >= cur_sec:
            # Add the index that the current second begins. Set it's key value
            # To be the corresponding second number.
            win_start_table[cur_sec//1000] = i
            cur_sec += 1000


        if some_data[i+1,0] > win_end :
            # Get 31 second window of data
            rows = win_start_table[win_start]
            x_win= some_data[rows:i+1, 1]
            y_win= some_data[rows:i+1, 2]
            z_win= some_data[rows:i+1, 3]
            mag_win= some_data[rows:i+1, 4]
            # Timestamp output
            output[output_iter, 0] = some_data[win_start_table[int(win_end//1000 - 15)],0]
            # Caclulate dominant frequency and power
            freq, power = dom_freq(mag_win)
            # Add calc to output
            output[output_iter, 1] = freq
            output[output_iter, 2] = power
            # Calculate MEAN
            output[output_iter, 3] = np.mean(mag_win)
            # Calculate STD
            output[output_iter, 4] = np.std(mag_win)
            # Coefficeint of Variation
            output[output_iter, 5] = output[output_iter, 4] / output[output_iter, 3]
            # Calculate Min
            output[output_iter, 6] = np.min(mag_win)
            # Calculate Max
            output[output_iter, 7] = np.max(mag_win)
            # Calculate 10th Percentile
            output[output_iter, 8] = np.percentile(mag_win, 10)
            # Calculate the 25th Percentile
            output[output_iter, 9] = np.percentile(mag_win, 25)
            # Calculate the 50th Percentile
            output[output_iter, 10] = np.percentile(mag_win, 50)
            # Calculate the 75th Percentile
            output[output_iter, 11] = np.percentile(mag_win, 75)
            # Calculate the 90th Percentile 
            output[output_iter, 12] = np.percentile(mag_win, 90)
            # Calculate the 3rd Moment
            output[output_iter, 13] = moment(mag_win, 3)
            # Calculate the 4th Moment
            output[output_iter, 14] = moment(mag_win, 4)
            # Calculate Skewness
            output[output_iter, 15] = skew(mag_win)
            # Calculate Kurtosis
            output[output_iter, 16] = kurtosis(mag_win)
            # Calculate the autocorrelation with second lag
            output[output_iter, 17] = acf(mag_win, nlags=50)[-1]
            # Correlate X with Y
            output[output_iter, 18] = np.correlate(x_win, y_win)
            # Correlate X with Z
            output[output_iter, 19] = np.correlate(x_win, z_win)
            # Correlate Y with Z
            output[output_iter, 20] = np.correlate(y_win, z_win)
            # Advance window by a second
            win_start += 1
            win_end += 1000
            output_iter += 1

    names = ['Time', 'Dominant Frequency', 'Power', 'Mag Mean', 'Mag STD', 'Mag CoV', 'Mag Min', 'Mag Max', 'Mag 10th Percentile',
               'Mag 25th Percentile', 'Mag 50th Percentile', 'Mag 75th Percentile', 'Mag 90th Percentile', 'Mag 3rd Moment',
               'Mag 4th Moment', 'Mag Skewness', 'Mag Kurtosis', 'Mag 1sec Autocorrelation', 'XY Correlation', 'XZ Correlation', 
               'YZ Correlation']
    out_df = pd.DataFrame(output, columns=names)
    out_df['Time'] = out_df['Time'].apply(lambda x: datetime.fromtimestamp((x+ start_time)//1000))
    if ds_begin <= out_df.iloc[0,0] <= ds_end:
        out_df['Time'] = out_df['Time'].apply(lambda x: x + timedelta(hours=4))
    else:
        out_df['Time'] = out_df['Time'].apply(lambda x: x + timedelta(hours=5))
    out_df.to_csv(out_path, index=False)

## RUN BELOW TO CALCULATE APPLE METRICS

In [37]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/ntindall/Wearables/Physical Activity/Rahul_Accelerometer_Metrics/Accel_Metrics/"


participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    apple_accel, start = read_apple(participant)
    if apple_accel is not None:
        # Define output path for metrics
        out_path = out_dir + participant_id + "_accel_metrics.csv"
        # Calculate Metrics and write to file
        accel_metrics(apple_accel, start, out_path)

    else:
        print(f"No Apple Data for {participant_id}")
    
    curr_participant += 1
    
    


Processing Participant 0118 1/162


C:\Users\ntindall\AppData\Local\Temp\ipykernel_520\3695143538.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


Processing Participant 0231 2/162


C:\Users\ntindall\AppData\Local\Temp\ipykernel_520\3695143538.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


Processing Participant 0444 3/162


...

Processing Participant 2820 160/162


C:\Users\ntindall\AppData\Local\Temp\ipykernel_520\3695143538.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


Processing Participant 2821 161/162


C:\Users\ntindall\AppData\Local\Temp\ipykernel_520\3695143538.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


Processing Participant 2824 162/162
No Apple Data for 2824


## RUN BELOW TO CALCULATE GARMIN METRICS

In [22]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/ntindall/Wearables/Physical Activity/Rahul_Accelerometer_Metrics/Garmin_Metrics/Accel_Metrics/"

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    garmin_accel, start = read_garmin(participant)
    if garmin_accel is not None:
        # Define output path for metrics
        out_path = out_dir + participant_id + "_accel_metrics.csv"
        # Calculate Metrics and write to file
        print(garmin_accel.shape)
        accel_metrics(garmin_accel, start, out_path)

    else:
        print(f"No Garmin Data for {participant_id}")
    
    curr_participant += 1

Processing Participant 0118 1/184
(105500, 5)
Processing Participant 0231 2/184
No Garmin Data for 0231
Processing Participant 0444 3/184
(100900, 5)
Processing Participant 0601 4/184
(107875, 5)
Processing Participant 0618 5/184
No Garmin Data for 0618
Processing Participant 0632 6/184
No Garmin Data for 0632
Processing Participant 0695 7/184
No Garmin Data for 0695
Processing Participant 0819 8/184
(113675, 5)
Processing Participant 0967 9/184
(92276, 5)
Processing Participant 1333 10/184
(99625, 5)
Processing Participant 1760 11/184
(101775, 5)
Processing Participant 1772 12/184
No Garmin Data for 1772
Processing Participant 1779 13/184
(97400, 5)
Processing Participant 1799 14/184
(101634, 5)
Processing Participant 1854 15/184
(93050, 5)
Processing Participant 1974 16/184
(105525, 5)
Processing Participant 1991 17/184
(176575, 5)
Processing Participant 2007 18/184
(94875, 5)
Processing Participant 2031 19/184
(108300, 5)
Processing Participant 2069 20/184
(113925, 5)
Processing Par

No Garmin Data for 2815
Processing Participant 2816 166/184
No Garmin Data for 2816
Processing Participant 2817 167/184
(93575, 5)
Processing Participant 2818 168/184
(105775, 5)
Processing Participant 2819 169/184
No Garmin Data for 2819
Processing Participant 2820 170/184
No Garmin Data for 2820
Processing Participant 2821 171/184
No Garmin Data for 2821
Processing Participant 2822 172/184
No Garmin Data for 2822
Processing Participant 2823 173/184
No Garmin Data for 2823
Processing Participant 2824 174/184
(117250, 5)
Processing Participant 2987 175/184
(95450, 5)
Processing Participant 2988 176/184
No Garmin Data for 2988
Processing Participant 2989 177/184
No Garmin Data for 2989
Processing Participant 2990 178/184
No Garmin Data for 2990
Processing Participant 2991 179/184
(99625, 5)
Processing Participant 2993 180/184
No Garmin Data for 2993
Processing Participant 2995 181/184
No Garmin Data for 2995
Processing Participant 2999 182/184
No Garmin Data for 2999
Processing Particip

## RUN BELOW TO CALCULATE FITBIT METRICS

In [11]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/ntindall/Wearables/Physical Activity/Rahul_Accelerometer_Metrics/Fitbit_Metrics/Accel_Metrics/"

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    fitbit_accel, start = read_fitbit(participant)
    if fitbit_accel is not None:
        # Define output path for metrics
        out_path = out_dir + participant_id + "_accel_metrics.csv"
        # Calculate Metrics and write to file
        print(fitbit_accel.shape)
        accel_metrics(fitbit_accel, start, out_path)

    else:
        print(f"No Fitbit Data for {participant_id}")
    
    curr_participant += 1

Processing Participant 0118 1/184
No Fitbit Data for 0118
Processing Participant 0231 2/184
(187789, 5)
Processing Participant 0444 3/184
No Fitbit Data for 0444
Processing Participant 0601 4/184
No Fitbit Data for 0601
Processing Participant 0618 5/184
(182548, 5)
Processing Participant 0632 6/184
(195933, 5)
Processing Participant 0695 7/184
(177747, 5)
Processing Participant 0819 8/184
No Fitbit Data for 0819
Processing Participant 0967 9/184
No Fitbit Data for 0967
Processing Participant 1333 10/184
(199402, 5)
Processing Participant 1760 11/184
(202971, 5)
Processing Participant 1772 12/184
(217207, 5)
Processing Participant 1779 13/184
No Fitbit Data for 1779
Processing Participant 1799 14/184
(191610, 5)
Processing Participant 1854 15/184
No Fitbit Data for 1854
Processing Participant 1974 16/184
No Fitbit Data for 1974
Processing Participant 1991 17/184
No Fitbit Data for 1991
Processing Participant 2007 18/184
No Fitbit Data for 2007
Processing Participant 2031 19/184
(211820,

Processing Participant 2783 150/184
No Fitbit Data for 2783
Processing Participant 2788 151/184
(181182, 5)
Processing Participant 2789 152/184
(182604, 5)
Processing Participant 2790 153/184
(196297, 5)
Processing Participant 2792 154/184
(183863, 5)
Processing Participant 2793 155/184
(183055, 5)
Processing Participant 2794 156/184
(176323, 5)
Processing Participant 2795 157/184
(223540, 5)
Processing Participant 2796 158/184
No Fitbit Data for 2796
Processing Participant 2797 159/184
(209604, 5)
Processing Participant 2798 160/184
(203859, 5)
Processing Participant 2801 161/184
(191415, 5)
Processing Participant 2812 162/184
(204360, 5)
Processing Participant 2813 163/184
(204194, 5)
Processing Participant 2814 164/184
(186944, 5)
Processing Participant 2815 165/184
(188222, 5)
Processing Participant 2816 166/184
(208606, 5)
Processing Participant 2817 167/184
(186585, 5)
Processing Participant 2818 168/184
(209776, 5)
Processing Participant 2819 169/184
No Fitbit Data for 2819
Proc